In [1]:
import re
from typing import List
import argparse

def text_to_stitches(text: str, punc_spaces: int=2, zero_as_punctuation: bool=True):
    stitches = ''
    
    for char in text.lower():
        
        if re.match(r'\s+', char):
            # all whitespace characters are 1:1 knit stitches
            stitches += 'k'
            
        elif re.match(r'\d+', char):
            # treates 0 as punctuation if flag is set
            if zero_as_punctuation and char == '0':
                    stitches += 'p'*punc_spaces + 'k'

            # all other digits are 1:1 purl stitches
            stitches += 'p' * int(char) + 'k'
            
        elif re.match(r'\w+', char):
            # all letters are 1:1 purl stitches
            stitches += 'p'

        else:
            # catchall, everything else is treated as punctuation and is 1:2 knit stitches
            stitches += 'k' * punc_spaces
            
    return stitches

def fix_long_repeats(string: str, 
                     max_purl_run_len: int=12, 
                     max_knit_run_len: int=2, 
                     purl_run_spacer_char: str='k'): 
    
    # Fix long purl runs
    pattern = r'(p)\1{' + str(max_purl_run_len) + r',}'
    matches = re.finditer(pattern, string)
    start_ixs = [match.start() for match in matches]
    
    for ix in start_ixs:
        string = string[:ix+max_purl_run_len] + purl_run_spacer_char + string[ix+max_purl_run_len:]
        
    # Fix long knit runs
    pattern = r'k{' + str(max_knit_run_len+1) + r',}'
    string = re.sub(pattern, 'k' * int(max_knit_run_len), string)
        
    return string


def break_sequence(sequence: str, grouping_size: int=10): 
    return [sequence[i:i+grouping_size] for i in range(0, len(sequence), grouping_size)]

def consolidate_sequence_into_pattern(sequence: str):
    consolidated = ''
    current = sequence[0]
    count = 1

    for stitch in sequence[1:]:
        if stitch == current:
            count += 1
        else:
            consolidated += (current + str(count) + ' ')
            current = stitch
            count = 1
    
    consolidated += (current + str(count) + ' ')
    
    return consolidated

def consolidate_sequences(sequences: List[str]):
    pattern = []
    for s in sequences:
        pattern.append(consolidate_sequence_into_pattern(s))
        
    return '\n'.join(pattern)

def text_to_knitting(text: str, 
                     stitches_per_line: int=10, 
                     max_purl_run_len: int=12, 
                     max_knit_run_len: int=2, 
                     purl_run_spacer_char: str='k', 
                     punctuation_mapping: int=2, 
                     treat_zero_as_punctuation: bool=True
                     ):
    
    stitches = text_to_stitches(text, 
                                punc_spaces=punctuation_mapping, 
                                zero_as_punctuation=treat_zero_as_punctuation
                                )
    
    stitches = fix_long_repeats(stitches, 
                                max_purl_run_len=max_purl_run_len, 
                                max_knit_run_len=max_knit_run_len,
                                purl_run_spacer_char=purl_run_spacer_char)
    
    stitch_groups = break_sequence(stitches, grouping_size=stitches_per_line)
    
    pattern = consolidate_sequences(stitch_groups)
    
    return pattern


def divide_original_text(text: str, grouping_size: int=10):
    return '\n'.join([text[i:i+grouping_size] for i in range(0, len(text), grouping_size)]).strip()

In [2]:
test_text = 'test helloworldsofwater how are you doing today 2024-05-03?'
pattern = text_to_knitting(test_text)    
print(pattern)

p4 k1 p5 
p7 k1 p2 
p4 k1 p3 k1 p1 
p2 k1 p3 k1 p3 
p2 k1 p5 k1 p1 
p1 k1 p2 k2 p2 k1 p1 
p3 k2 p2 k2 p1 
p4 k2 p2 k2 
p3 k2 


In [3]:
texts = ['''I never felt magic crazy as this
I never saw moons, knew the meaning of the sea
I never held emotion in the palm of my hand
Or felt sweet breezes in the top of a tree
But now you're here
Brighten my northern sky
I've been a long time that I'm waiting
Been a long that I'm blown
I've been a long time that I've wandered
Through the people I have known
Oh, if you would and you could
Straighten my new mind's eye
Would you love me for my money?
Would you love me for my head?
Would you love me through the winter?
Would you love me 'til I'm dead?
Oh, if you would and you could
Come blow your horn on high
I never felt magic crazy as this
I never saw moons, knew the meaning of the sea
I never held emotion in the palm of my hand
Or felt sweet breezes in the top of a tree
But now you're here
Brighten my northern sky
'''
, '''
Pray God you can cope
I stand outside this woman's work
This woman's world
Ooh, it's hard on the man
Now his part is over
Now starts the craft of the father
I know you have a little life in you yet
I know you have a lot of strength left
I know you have a little life in you yet
I know you have a lot of strength left
I should be crying, but I just can't let it show
I should be hoping, but I can't stop thinking
Of all the things I should've said
That I never said
All the things we should've done
That we never did
All the things I should've given
But I didn't
Oh, darling, make it go
Make it go away
Give me these moments back
Give them back to me
Give me that little kiss
Give me your hand
I know you have a little life in you yet
I know you have a lot of strength left
I know you have a little life in you yet
I know you have a lot of strength left
I should be crying, but I just can't let it show
I should be hoping, but I can't stop thinking
Of all the things we should've said
That were never said
All the things we should've done
That we never did
All the things that you needed from me
All the things that you wanted for me
All the things that I should've given but I didn't
Oh, darling, make it go away
Just make it go away now
'''
, '''
My life has been a tapestry
Of rich and royal hue
An everlasting vision
Of the ever-changing view
A wond'rous woven magic
In bits of blue and gold
A tapestry to feel and see
Impossible to hold
Once amid the soft silver
Sadness in the sky
There came a man of fortune
A drifter passing by
He wore a torn and tattered cloth
Around his leathered hide
And a coat of many colors
Yellow, green, on either side
He moved with some uncertainty
As if he didn't know
Just what he was there for
Or where he ought to go
Once he reached for something
Golden hanging from a tree
And his hand came down empty
Soon within my tapestry
Along the rutted road
He sat down on a river rock
And turned into a toad
It seemed that he had fallen
Into someone's wicked spell
And I wept to see him suffer
Though I didn't know him well
As I watched in sorrow
There suddenly appeared
A figure gray and ghostly
Beneath a flowing beard
In times of deepest darkness
I've seen him dressed in black
Now my tapestry's unraveling
He's come to take me back
He's come to take me back
'''
]

spacer_sequence = 'a 1 2 3 4 '
# spacer_sequence = '  x x x x x x  '

text = spacer_sequence.join(texts)

In [4]:
pattern = text_to_knitting(text)    
print(pattern)

p1 k1 p5 k1 p2 
p2 k1 p5 k1 p1 
p4 k1 p2 k1 p2 
p2 k1 p1 k1 p5 
k1 p3 k1 p5 
k2 p4 k1 p3 
k1 p7 k1 p1 
p1 k1 p3 k1 p3 k1 
p1 k1 p5 k1 p2 
p2 k1 p7 
k1 p2 k1 p3 k1 p2 
p2 k1 p2 k1 p2 k1 p1 
p3 k1 p2 k1 p3 
p1 k1 p5 k1 p2 
p5 k1 p2 k1 p1 
p2 k1 p3 k1 p2 k1 
p1 k1 p4 k1 p3 
k1 p3 k1 p3 k2 
p2 k1 p4 k1 p2 
p6 k1 p2 k1 
p8 k1 p1 
p2 k1 p1 k2 p2 k1 p1 
p3 k1 p1 k1 p4 
k1 p4 k1 p4 
k1 p1 k2 p1 k1 p4 
p3 k1 p4 k1 p1 
k1 p4 k1 p4 
k1 p1 k2 p1 k1 p4 
p1 k1 p1 k2 p2 k1 p2 
p2 k1 p1 k1 p4 k1 
p4 k1 p4 k1 
p1 k2 p2 k1 p4 
p4 k1 p5 
p2 k1 p3 k1 p3 
p3 k1 p1 k1 p4 
k1 p5 k1 p2 k1 
k1 p2 k1 p3 k1 p2 
p3 k1 p3 k1 p2 
p1 k1 p5 k1 p2 
p8 k1 p1 
p1 k1 p3 k1 p4 
k2 p1 k1 p3 k1 p2 
p3 k1 p3 k1 p2 
p2 k1 p2 k1 p3 k1 
p2 k1 p5 k2 
p5 k1 p3 k1 
p4 k1 p2 k1 p2 
p1 k1 p2 k1 p4 k1 
k1 p5 k1 p3 
k1 p4 k1 p2 k1 p1 
p6 k1 p3 
k1 p6 k2 p1 
p4 k1 p3 k1 p1 
p3 k1 p2 k2 p2 
p1 k1 p1 k2 p1 k1 p3 
p1 k2 p2 k2 p2 k1 
p3 k1 p5 k1 
p3 k1 p3 k1 p2 
p3 k1 p4 k1 p1 
p3 k1 p4 k1 p1 
p3 k1 p2 k1 p3 
p1 k1 p1 k1 p5 k1 
p4 k1 p5 
k